<a href="https://colab.research.google.com/github/cbysousa/imersao-ia/blob/main/sistema_de_busca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U -q google-generativeai

In [5]:
import numpy as np      #biblioteca específica para vetores/cálculos
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("SK_A5")
genai.configure(api_key = api_key)

In [6]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:      #converte texto (tokens) em valores numéricos
    print(m.name)

models/embedding-001
models/text-embedding-004


In [7]:
#Exemplo de embedding
title = "A próxima geraçãp de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo copmpleto:\n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model = "models/embedding-001",
                                   content = sample_text,
                                   title = title,
                                   task_type = "RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.047649696, -0.037753318, -0.022145621, 0.02023632, 0.06742927, -0.002176867, -0.027060792, -0.018205767, 0.057370692, 0.06300416, 0.013718054, 0.014801277, -0.044553943, -0.02083795, 0.0100287935, -0.020240838, 0.025657209, -0.007701206, -0.03736588, -0.0037014005, 0.010353611, 0.009674645, -0.028793106, -0.06459184, -0.020650748, 0.015003403, 0.010954744, -0.036441788, -0.036419, 0.023957074, -0.051779374, 0.049785417, -0.033260796, 0.014760996, -0.039400086, -0.036315847, -0.03042126, -0.04051059, -0.0009133958, -0.005301281, 0.004295843, -0.08520947, -0.013713355, 0.024400223, -0.0018780497, -0.024863016, 0.043373823, 0.04780855, 0.013209713, -0.054273516, 0.026521048, 0.01843939, 0.06962168, -0.04044223, -0.0015372969, -0.013443345, 0.02081667, -0.024535833, 0.023657825, -0.000299423, -0.0053715985, 0.023568228, -0.014187812, 0.054119274, 0.020164248, -0.059904817, -0.047991063, -0.004376142, 0.009722481, 0.045533102, 0.0052371314, 0.01339833, 0.055765238, -0.06278

In [8]:
#Listagem de documentos que serão usados
DOCUMENT1 = {
    "Título": "Dorival ainda está em busca do “modo de usar” Vini Jr",
    "Conteúdo": "Nenhuma surpresa na lista de Dorival Jr. A volta de alguns veteranos que estavam lesionados na outra convocação estava dentro do script. Dorival já tem o seu time de confiança para a Copa América, mas um dos seus maiores desafios nessa fase atual da Seleção Brasileira é conseguir o Manual de Instruções como o “Modo de Usar” Vinícius Jr."
}
DOCUMENT2 = {
    "Título": "Al-Hilal, de Jorge Jesus, goleia Al-Hazem e é campeão saudita pela 19ª vez",
    "Conteúdo": "Mais um campeonato definido em 2023/24. Com a goleada por 4 a 1 sobre o Al-Hazem, o Al-Hilal conquistou o Campeonato Saudita com três rodadas de antecedência. Este é o 19º título nacional da equipe de Jorge Jesus."
}
DOCUMENT3 = {
    "Título": "Ministro do Esporte pede à CBF paralisação do Campeonato Brasileiro de futebol",
    "Conteúdo": "O pedido de paralisação total do Brasileirão foi apresentado em razão das fortes chuvas que atingem o Rio Grande do Sul. Nesta quinta-feira (9), Fufuca já havia informado que pediria a paralisação da competição para que as atenções do país se voltem à situação do povo gaúcho. "
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [9]:
df = pd.DataFrame(documents)
df.columns = ["titulo", "conteudo"]     #função de mudar o nome das colunas
df

,titulo,conteudo
0,Dorival ainda está em busca do “modo de usar” ...,Nenhuma surpresa na lista de Dorival Jr. A vol...
1,"Al-Hilal, de Jorge Jesus, goleia Al-Hazem e é ...",Mais um campeonato definido em 2023/24. Com a ...
2,Ministro do Esporte pede à CBF paralisação do ...,O pedido de paralisação total do Brasileirão f...


In [10]:
model = "models/embedding-001"

In [11]:
def embed_fn(title, text):
  return genai.embed_content(model = model,
                            content = text,
                            title = title,
                            task_type = "RETRIEVAL_DOCUMENT") ["embedding"]

In [12]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["titulo"], row["conteudo"]), axis = 1)
df

,titulo,conteudo,Embeddings
0,Dorival ainda está em busca do “modo de usar” ...,Nenhuma surpresa na lista de Dorival Jr. A vol...,"[-0.00417222, -0.016628997, -0.045169935, -0.0..."
1,"Al-Hilal, de Jorge Jesus, goleia Al-Hazem e é ...",Mais um campeonato definido em 2023/24. Com a ...,"[0.014686327, -0.017386878, -0.045812506, 0.03..."
2,Ministro do Esporte pede à CBF paralisação do ...,O pedido de paralisação total do Brasileirão f...,"[-0.0011712428, -0.025369024, -0.07856552, 0.0..."


In [26]:
import numpy as np

def gerar_e_buscar_consulta(consulta, df, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                                content=consulta,
                                                task_type="RETRIEVAL_QUERY")["embedding"]     # Acessar o array do embedding

    produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["conteudo"]

In [ ]:
consulta = "Como está a situação do brasileirão?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

In [28]:
generation_config = {
    "temperature": 0.5,
    "candidate_count":1
}

In [29]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)

Galera, vai rolar uma parada geral no Brasileirão por causa da chuva que tá caindo forte no Rio Grande do Sul. O Fufuca já deu a letra ontem que ia pedir uma pausa na bolinha pra geral ficar de olho na situação do pessoal que tá precisando por lá.
